# Export model and eval

In [1]:
control_yml = r'e:\PaddlePaddle\PaddleDetection\configs\ppyoloe\ppyoloe_plus_crn_m_custom.yml'
# best_model_path = r'e:\PaddlePaddle\PaddleDetection\output\ppyoloe_plus_crn_l_custom_disease\69.pdparams'
best_model_path = r'e:\PaddlePaddle\PaddleDetection\output\ppyoloe_plus_crn_m_custom\178.pdparams'
# best_model_path = r'e:\PaddlePaddle\PaddleDetection\output\ppyoloe_plus_crn_m_custom\best_model.pdparams'
inference_model = r"e:\PaddlePaddle\PaddleDetection\output_inference\ppyoloe_plus_crn_m_custom"


In [4]:
# export model for trt
!python tools/export_model.py -c $control_yml -o weights=$best_model_path trt=True exclude_nms=True TestReader.inputs_def.image_shape=[3,512,512] 

[05/12 15:59:10] ppdet.utils.checkpoint INFO: Finish loading model weights: e:\PaddlePaddle\PaddleDetection\output\ppyoloe_plus_crn_m_custom\178.pdparams
[05/12 15:59:11] ppdet.engine INFO: Export inference config file to output_inference\ppyoloe_plus_crn_m_custom\infer_cfg.yml
[05/12 15:59:20] ppdet.engine INFO: Export model and saved in output_inference\ppyoloe_plus_crn_m_custom


In [5]:
# for inference
!python tools/export_model.py -c $control_yml -o weights=$best_model_path TestReader.inputs_def.image_shape=[3,512,512]

[05/12 15:59:59] ppdet.utils.checkpoint INFO: Finish loading model weights: e:\PaddlePaddle\PaddleDetection\output\ppyoloe_plus_crn_m_custom\178.pdparams
[05/12 16:00:00] ppdet.engine INFO: Export inference config file to output_inference\ppyoloe_plus_crn_m_custom\infer_cfg.yml
[05/12 16:00:07] ppdet.engine INFO: Export model and saved in output_inference\ppyoloe_plus_crn_m_custom


In [2]:
# eval
!python tools/eval.py -c $control_yml -o weights=$best_model_path TestReader.inputs_def.image_shape=[3,512,512] --classwise --match_metric iou

[05/12 15:56:56] ppdet.utils.checkpoint INFO: Finish loading model weights: e:\PaddlePaddle\PaddleDetection\output\ppyoloe_plus_crn_m_custom\178.pdparams
[05/12 15:56:59] ppdet.engine INFO: Eval iter: 0
[05/12 15:57:03] ppdet.metrics.metrics INFO: Accumulating evaluatation results...
[05/12 15:57:04] ppdet.metrics.map_utils INFO: Per-category of VOC AP: 
+-----------+-------+---------------+-------+----------+-------+
| category  | AP    | category      | AP    | category | AP    |
+-----------+-------+---------------+-------+----------+-------+
| LineCrack | 0.558 | AligatorCrack | 0.311 | Repair   | 0.477 |
| Pothole   | 0.729 | None          | None  | None     | None  |
+-----------+-------+---------------+-------+----------+-------+
[05/12 15:57:04] ppdet.metrics.map_utils INFO: per-category PR curve has output to voc_pr_curve folder.
[05/12 15:57:04] ppdet.metrics.metrics INFO: mAP(0.50, 11point) = 51.87%
[05/12 15:57:04] ppdet.engine INFO: Total sample number: 168, average FPS: 2

W0512 15:56:53.906421  7952 gpu_resources.cc:61] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 11.7, Runtime API Version: 11.6
W0512 15:56:53.906421  7952 gpu_resources.cc:91] device: 0, cuDNN Version: 8.8.


In [3]:
# eval
!python tools/eval.py -c $control_yml -o weights=e:\PaddlePaddle\PaddleDetection\output\ppyoloe_plus_crn_m_custom\best_model.pdparams TestReader.inputs_def.image_shape=[3,512,512] --classwise --match_metric iou

[05/12 15:58:22] ppdet.utils.checkpoint INFO: Finish loading model weights: e:\PaddlePaddle\PaddleDetection\output\ppyoloe_plus_crn_m_custom\best_model.pdparams
[05/12 15:58:25] ppdet.engine INFO: Eval iter: 0
[05/12 15:58:29] ppdet.metrics.metrics INFO: Accumulating evaluatation results...
[05/12 15:58:30] ppdet.metrics.map_utils INFO: Per-category of VOC AP: 
+-----------+-------+---------------+-------+----------+-------+
| category  | AP    | category      | AP    | category | AP    |
+-----------+-------+---------------+-------+----------+-------+
| LineCrack | 0.639 | AligatorCrack | 0.341 | Repair   | 0.558 |
| Pothole   | 0.784 | None          | None  | None     | None  |
+-----------+-------+---------------+-------+----------+-------+
[05/12 15:58:30] ppdet.metrics.map_utils INFO: per-category PR curve has output to voc_pr_curve folder.
[05/12 15:58:30] ppdet.metrics.metrics INFO: mAP(0.50, 11point) = 58.04%
[05/12 15:58:30] ppdet.engine INFO: Total sample number: 168, average

W0512 15:58:19.954591  9920 gpu_resources.cc:61] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 11.7, Runtime API Version: 11.6
W0512 15:58:19.954591  9920 gpu_resources.cc:91] device: 0, cuDNN Version: 8.8.


# generate images for inference

In [6]:
import numpy as np
import os
import shutil
import glob

image_path = r'dataset\disease\images'
test_path = r'dataset\disease\test_images'
if os.path.exists(test_path):
    shutil.rmtree(test_path)
os.makedirs(test_path, exist_ok=True)
imgs = glob.glob(os.path.join(image_path, '**', '*.jpg'), recursive=True)
infer_imgs = np.random.choice(imgs, 10)
for img in infer_imgs:
    img_name = os.path.basename(img)
    src_path = img
    dst_path = os.path.join(test_path, img_name)
    shutil.copy(src_path, dst_path)

# Inference

In [7]:
# infer with dynamic graphs
import os
if os.path.exists("results"):
    shutil.rmtree("results")
!python tools/infer.py -c $control_yml -o weights=$best_model_path --infer_dir=$test_path --output_dir=results

[05/12 16:04:56] ppdet.utils.checkpoint INFO: Finish loading model weights: e:\PaddlePaddle\PaddleDetection\output\ppyoloe_plus_crn_m_custom\178.pdparams
[05/12 16:04:56] train INFO: Found 10 inference images in total.
[05/12 16:04:58] ppdet.engine INFO: Detection bbox results save in results\109d0d78e7294d1010ead9e87f7096d5.jpg
[05/12 16:04:58] ppdet.engine INFO: Detection bbox results save in results\2a92b51749b90f12b768781d2213c530.jpg
[05/12 16:04:58] ppdet.engine INFO: Detection bbox results save in results\165f3d4735f58631f6b576e955c4d4a6.jpg
[05/12 16:04:58] ppdet.engine INFO: Detection bbox results save in results\2f16f050329b3bf0364dc8836e3035cc.jpg
[05/12 16:04:58] ppdet.engine INFO: Detection bbox results save in results\7f76b0385fe59a6a5bce536bf60e92fb.jpg
[05/12 16:04:58] ppdet.engine INFO: Detection bbox results save in results\af8bdd916639e80d06d31411d46f7167.jpg
[05/12 16:04:58] ppdet.engine INFO: Detection bbox results save in results\fcc271696d86a360f4176593501d5f78.j

W0512 16:04:54.323982 10352 gpu_resources.cc:61] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 11.7, Runtime API Version: 11.6
W0512 16:04:54.323982 10352 gpu_resources.cc:91] device: 0, cuDNN Version: 8.8.

100%|██████████| 10/10 [00:01<00:00,  5.12it/s]


In [6]:
# infer with dynamic graphs

# !python deploy/python/infer.py --model_dir=$inference_model --image_dir=$test_path --device=GPU --run_mode=trt_fp16 --output_dir=results_trt
if os.path.exists("results"):
    shutil.rmtree("results")
!python deploy/python/infer.py --model_dir=$inference_model --image_dir=$test_path --device=CPU --output_dir=results

-----------  Running Arguments -----------
action_file: None
batch_size: 1
camera_id: -1
combine_method: nms
cpu_threads: 1
device: CPU
enable_mkldnn: False
enable_mkldnn_bfloat16: False
image_dir: dataset\disease\test_images
image_file: None
match_metric: ios
match_threshold: 0.6
model_dir: e:\PaddlePaddle\PaddleDetection\output_inference\ppyoloe_plus_crn_m_custom
output_dir: results
overlap_ratio: [0.25, 0.25]
random_pad: False
reid_batch_size: 50
reid_model_dir: None
run_benchmark: False
run_mode: paddle
save_images: True
save_mot_txt_per_img: False
save_mot_txts: False
save_results: False
scaled: False
slice_infer: False
slice_size: [640, 640]
threshold: 0.5
tracker_config: None
trt_calib_mode: False
trt_max_shape: 1280
trt_min_shape: 1
trt_opt_shape: 640
use_coco_category: False
use_dark: True
use_gpu: False
video_file: None
window_size: 50
------------------------------------------
-----------  Model Configuration -----------
Model Arch: YOLO
Transform Order: 
--transform op: Res

e:\PaddlePaddle\PaddleDetection\deploy\python\visualize.py:162: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  tw, th = draw.textsize(text)
e:\PaddlePaddle\PaddleDetection\deploy\python\visualize.py:162: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  tw, th = draw.textsize(text)
e:\PaddlePaddle\PaddleDetection\deploy\python\visualize.py:162: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  tw, th = draw.textsize(text)
e:\PaddlePaddle\PaddleDetection\deploy\python\visualize.py:162: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  tw, th = draw.textsize(text)
e:\PaddlePaddle\PaddleDetection\deploy\python\visualize.py:162: DeprecationWarning: textsize is deprecated and will 

# ONNX

In [9]:
# Export 
!paddle2onnx --model_dir output_inference/ppyoloe_plus_crn_m_custom_std_trt --model_filename model.pdmodel --params_filename model.pdiparams --save_file onnx_model/ppyoloe_plus_crn_m_custom_std_trt.onnx --input_shape_dict "{'image':[1, 3, 512, 512], 'scale_factor': [1, 2]}"

2023-05-12 16:06:27 [WARNING]	[Deprecated] The flag `--input_shape_dict` is deprecated, if you need to modify the input shape of PaddlePaddle model, please refer to this tool https://github.com/jiangjiajun/PaddleUtils/tree/main/paddle 
[Paddle2ONNX] Start to parse PaddlePaddle model...
[Paddle2ONNX] Model file path: output_inference/ppyoloe_plus_crn_m_custom_std_trt\model.pdmodel
[Paddle2ONNX] Paramters file path: output_inference/ppyoloe_plus_crn_m_custom_std_trt\model.pdiparams
[Paddle2ONNX] Start to parsing Paddle model...
[Paddle2ONNX] [nearest_interp_v2: nearest_interp_v2_0.tmp_0] Requires the minimal opset version of 11.
[Paddle2ONNX] [nearest_interp_v2: nearest_interp_v2_1.tmp_0] Requires the minimal opset version of 11.
[Paddle2ONNX] Due to the operator: nearest_interp_v2, requires opset_version >= 11.
[Paddle2ONNX] Opset version will change to 11 from 9
[Paddle2ONNX] Use opset_version = 11 for ONNX export.
[Paddle2ONNX] PaddlePaddle model is exported as ONNX format now.
2023-0

In [10]:
import onnx
import onnxsim
import numpy as np
import onnx_graphsurgeon as gs
from onnx import shape_inference
from collections import OrderedDict
import os


## Inference Using ONNX

In [11]:
import onnxruntime as ort
import numpy as np
import cv2
images = glob.glob(os.path.join(test_path, '**', '*.jpg'), recursive=True)
model = ort.InferenceSession("onnx_model/ppyoloe_plus_crn_m_custom_std.onnx", providers=["CUDAExecutionProvider"])

In [12]:
import onnxruntime as rt
import cv2
import numpy as np
class BaseOnnxInfer(object):
    def __init__(self, onnx_path, imgsz=(480,480)):
        self.imgsz = imgsz
        self.sess = ort.InferenceSession(onnx_path, providers=["CUDAExecutionProvider"])
        
    def predict(self, img,threshold):
        self.img = self.preprocess(img)
        data = self.sess.run(["multiclass_nms3_0.tmp_0","multiclass_nms3_0.tmp_2"], {'image': self.img, 'scale_factor': np.array([self.scale_factor]).astype('float32')})
        results = self.postprocess(data, threshold)
        return results

    def letterbox(self,im,color=(114, 114, 114), auto=False, scaleup=True, stride=32):
        # Resize and pad image while meeting stride-multiple constraints
        shape = im.shape[:2]  # current shape [height, width]
        new_shape = self.imgsz
        if isinstance(new_shape, int):
            new_shape = (new_shape, new_shape)
        # Scale ratio (new / old)
        self.scale_factor = new_shape[0] / shape[0], new_shape[1] / shape[1]
        self.img = cv2.resize(im, new_shape)  # add border
        return self.img

    def preprocess(self,image):
        self.img = self.letterbox(image)
        self.img = cv2.cvtColor(self.img,cv2.COLOR_BGR2RGB)
        self.img = self.img.astype(np.float32)
        self.img = self.img / 255.
        self.img -= np.array([0.485, 0.456, 0.406])[None, None, :]
        self.img /= np.array([0.229, 0.224, 0.225])[None, None, :]
        self.img = self.img.transpose((2, 0, 1))
        self.img = np.expand_dims(self.img,0)
        return self.img

    def postprocess(self,pred,threshold):
        new_bboxes = []
        for _pred in pred[0]:
            clas = _pred[0]
            score = _pred[1]
            if(score < threshold):
                continue
            bboxes = _pred[2:6]
            # xmin = (bboxes[0] - self.dw)/self.r
            # ymin = (bboxes[1] - self.dh)/self.r
            # xmax = (bboxes[2] - self.dw)/self.r
            # ymax = (bboxes[3] - self.dh)/self.r
            xmin, ymin, xmax, ymax = bboxes
            new_bboxes.append([clas, score, xmin,ymin,xmax,ymax])
        return new_bboxes


def visualize(img,bbox_array):
    cc_map = {0:("LineCrack", (255,np.random.randint(0,255),np.random.randint(0,255))),
              1:("AligatorCrack", (255,np.random.randint(0,255),np.random.randint(0,255))),
              2:("Repair", (255,np.random.randint(0,255),np.random.randint(0,255))),
              3:("Pothole", (255,np.random.randint(0,255),np.random.randint(0,255)))}
    for temp in bbox_array:
        xmin = int(temp[2])
        ymin = int(temp[3])
        xmax = int(temp[4])
        ymax = int(temp[5])
        clas = int(temp[0])
        score = temp[1]

        """
        LineCrack
        AligatorCrack
        Repair
        Pothole

        """
        clas, color = cc_map[clas]
        cv2.rectangle(img,(xmin,ymin),(xmax,ymax), color, 2)
        img = cv2.putText(img, "class:"+str(clas)+" "+str(round(score,2)), (xmin,int(ymin)+20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
    return img

sess = BaseOnnxInfer("onnx_model/ppyoloe_plus_crn_m_custom_std.onnx", imgsz=(512,512))
img1 = cv2.imread(r"dataset\rdd\test_images\China_Drone_000621.jpg")
results = sess.predict(img1,threshold=0.5)
img = visualize(img1,results)

cv2.imwrite("results/result.png", img)

True

## SpeedTest

In [13]:
%timeit sess.predict(img1,threshold=0.5)

25.6 ms ± 545 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Formal Inference

In [15]:
if os.path.exists("results_onnx"):
    shutil.rmtree("results_onnx")
os.makedirs("results_onnx", exist_ok=True)
trt_engine = BaseOnnxInfer("onnx_model/ppyoloe_plus_crn_m_custom_std.onnx", imgsz=(512,512))
for img in glob.glob(f"{test_path}/*.jpg"):
    img1 = cv2.imread(img)
    results = trt_engine.predict(img1,threshold=0.4)
    img1 = visualize(img1,results)
    cv2.imwrite("results_onnx/"+os.path.basename(img),img1)

## Surgery

In [17]:
# Place EfficientNMS for TensorRT
# 注意修改
########################################################
INPUT_PATH = './onnx_model/ppyoloe_plus_crn_m_custom_std_trt.onnx'
WEIGHTS_TYPE = "l"
SAVE_PATH = './onnx_nms_model/ppyoloe_plus_crn_m_custom_std_trt_nms.onnx'
CLASS_NUM = 4
SCORE_THRESHOLD = 0.25
IOU_THRESHOLD = 0.45
########################################################

gs_graph = gs.import_onnx(onnx.load(INPUT_PATH))
# fold constants
gs_graph.fold_constants()
gs_graph.cleanup().toposort()


Mul = [node for node in gs_graph.nodes if node.name=='p2o.Mul.200'][0]
Concat_14 = [node for node in gs_graph.nodes if node.name=='p2o.Concat.28'][0]
Mul_linear = 5376

scores = gs.Variable(name='scores',shape=[1,Mul_linear,CLASS_NUM],dtype=np.float32)
Transpose = gs.Node(name='lastTranspose',op='Transpose',
                   inputs=[Concat_14.outputs[0]],
                   outputs=[scores],
                   attrs=OrderedDict(perm=[0,2,1]))
gs_graph.nodes.append(Transpose)

Mul.outputs[0].name = 'boxes'
gs_graph.inputs = [gs_graph.inputs[0]]
gs_graph.outputs = [Mul.outputs[0],scores]
gs_graph.outputs[0].dtype=np.float32
gs_graph.outputs[1].dtype=np.float32

gs_graph.cleanup().toposort()
onnx_graph = shape_inference.infer_shapes(gs.export_onnx(gs_graph))
onnx_graph, check = onnxsim.simplify(onnx_graph)

gs_graph = gs.import_onnx(onnx_graph)
op_inputs = gs_graph.outputs
op = "EfficientNMS_TRT"
attrs = {
    "plugin_version": "1",
    "background_class": -1,
    "max_output_boxes": 100,
    "score_threshold": SCORE_THRESHOLD,
    "iou_threshold": IOU_THRESHOLD,
    "score_activation": False,
    "box_coding": 0,
}

output_num_detections = gs.Variable(
    name="num_dets",
    dtype=np.int32,
    shape=[1, 1],
)
output_boxes = gs.Variable(
    name="det_boxes",
    dtype=np.float32,
    shape=[1, 100, 4],
)
output_scores = gs.Variable(
    name="det_scores",
    dtype=np.float32,
    shape=[1, 100],
)
output_labels = gs.Variable(
    name="det_classes",
    dtype=np.int32,
    shape=[1, 100],
)
op_outputs = [
    output_num_detections, output_boxes, output_scores, output_labels
]

TRT = gs.Node(op=op,name="batched_nms",inputs=op_inputs,outputs=op_outputs,attrs=attrs)
gs_graph.nodes.append(TRT)
gs_graph.outputs = op_outputs
gs_graph.cleanup().toposort()

onnx.save(gs.export_onnx(gs_graph),SAVE_PATH)
print("finished")


finished


TensorRT
```bash
trtexec.exe --onnx=e:\PaddlePaddle\PaddleDetection\onnx_nms_model\ppyoloe_plus_crn_m_custom_std_trt_nms.onnx --saveEngine=e:\PaddlePaddle\PaddleDetection\onnx_nms_model\ppyoloe_plus_crn_m_custom_std_trt_nms.engine --fp16
```

# get engine and Inference

In [48]:
import tensorrt as trt
import pycuda.autoinit
import pycuda.driver as cuda
import numpy as np
import cv2
import time

class BaseEngine(object):
    def __init__(self, engine_path, imgsz=(480,480)):
        self.imgsz = imgsz
        logger = trt.Logger(trt.Logger.WARNING)
        runtime = trt.Runtime(logger)
        trt.init_libnvinfer_plugins(logger, namespace="")
        with open(engine_path, "rb") as f:
            serialized_engine = f.read()
        engine = runtime.deserialize_cuda_engine(serialized_engine)
        self.context = engine.create_execution_context()
        self.inputs, self.outputs, self.bindings = [], [], []
        self.stream = cuda.Stream()
        for binding in engine:
            size = trt.volume(engine.get_binding_shape(binding))
            dtype = trt.nptype(engine.get_binding_dtype(binding))
            host_mem = cuda.pagelocked_empty(size, dtype)
            device_mem = cuda.mem_alloc(host_mem.nbytes)
            self.bindings.append(int(device_mem))
            if engine.binding_is_input(binding):
                self.inputs.append({'host': host_mem, 'device': device_mem})
            else:
                self.outputs.append({'host': host_mem, 'device': device_mem})        

    def predict(self, img,threshold):
        self.img = self.preprocess(img)
        self.inputs[0]['host'] = np.ravel(self.img)
        # transfer data to the gpu
        for inp in self.inputs:
            cuda.memcpy_htod_async(inp['device'], inp['host'], self.stream)
        # run inference
        self.context.execute_async_v2(
            bindings=self.bindings,
            stream_handle=self.stream.handle)
        # fetch outputs from gpu
        for out in self.outputs:
            cuda.memcpy_dtoh_async(out['host'], out['device'], self.stream)
        # synchronize stream
        self.stream.synchronize()

        data = [out['host'] for out in self.outputs]
        results = self.postprocess(data,threshold)
        return results

    def letterbox(self,im,color=(114, 114, 114), auto=False, scaleup=True, stride=32):
        # Resize and pad image while meeting stride-multiple constraints
        shape = im.shape[:2]  # current shape [height, width]
        new_shape = self.imgsz
        if isinstance(new_shape, int):
            new_shape = (new_shape, new_shape)
        # Scale ratio (new / old)
        self.r = min(new_shape[0] / shape[0], new_shape[1] / shape[1])
        if not scaleup:  # only scale down, do not scale up (for better val mAP)
            self.r = min(self.r, 1.0)
        # Compute padding
        new_unpad = int(round(shape[1] * self.r)), int(round(shape[0] * self.r))
        self.dw, self.dh = new_shape[1] - new_unpad[0], new_shape[0] - new_unpad[1]  # wh padding
        if auto:  # minimum rectangle
            self.dw, self.dh = np.mod(self.dw, stride), np.mod(self.dh, stride)  # wh padding
        self.dw /= 2  # divide padding into 2 sides
        self.dh /= 2
        if shape[::-1] != new_unpad:  # resize
            im = cv2.resize(im, new_unpad, interpolation=cv2.INTER_LINEAR)
        top, bottom = int(round(self.dh - 0.1)), int(round(self.dh + 0.1))
        left, right = int(round(self.dw - 0.1)), int(round(self.dw + 0.1))
        self.img = cv2.copyMakeBorder(im, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)  # add border
        return self.img,self.r,self.dw,self.dh

    def preprocess(self,image):
        self.img,self.r,self.dw,self.dh = self.letterbox(image)
        self.img = cv2.cvtColor(self.img,cv2.COLOR_BGR2RGB)
        self.img = self.img.astype(np.float32)
        self.img = self.img / 255.
        self.img -= np.array([0.485, 0.456, 0.406])[None, None, :]
        self.img /= np.array([0.229, 0.224, 0.225])[None, None, :]
        self.img = self.img.transpose((2, 0, 1))
        self.img = np.expand_dims(self.img,0)
        return self.img

    def postprocess(self,pred,threshold):
        new_bboxes = []
        num =int(pred[0][0])
        bboxes = pred[1].reshape(-1,4)
        scores = pred[2]
        clas = pred[3]
        for i in range(num):
            if(scores[i] < threshold):
                continue
            xmin = (bboxes[i][0] - self.dw)/self.r
            ymin = (bboxes[i][1] - self.dh)/self.r
            xmax = (bboxes[i][2] - self.dw)/self.r
            ymax = (bboxes[i][3] - self.dh)/self.r
            new_bboxes.append([clas[i],scores[i],xmin,ymin,xmax,ymax])
        return new_bboxes


def visualize(img,bbox_array):
    cc_map = {0:("LineCrack", (255,np.random.randint(0,255),np.random.randint(0,255))),
              1:("AligatorCrack", (255,np.random.randint(0,255),np.random.randint(0,255))),
              2:("Repair", (255,np.random.randint(0,255),np.random.randint(0,255))),
              3:("Pothole", (255,np.random.randint(0,255),np.random.randint(0,255)))}
    for temp in bbox_array:
        xmin = int(temp[2])
        ymin = int(temp[3])
        xmax = int(temp[4])
        ymax = int(temp[5])
        clas = int(temp[0])
        score = temp[1]

        """
        LineCrack
        AligatorCrack
        Repair
        Pothole

        """
        clas, color = cc_map[clas]
        cv2.rectangle(img,(xmin,ymin),(xmax,ymax), color, 2)
        img = cv2.putText(img, "class:"+str(clas)+" "+str(round(score,2)), (xmin,int(ymin)+20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
    return img

trt_engine = BaseEngine("onnx_nms_model/ppyoloe_plus_crn_m_custom_std_trt_nms.engine", imgsz=(512,512))
img1 = cv2.imread(r"dataset\rdd\test_images\China_Drone_000621.jpg")
results = trt_engine.predict(img1,threshold=0.5)
img = visualize(img1,results)

cv2.imwrite("results_trt/China_Drone_000621.jpg",img)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_8232\98667936.py:21: DeprecationWarning: Use get_tensor_shape instead.
  size = trt.volume(engine.get_binding_shape(binding))
C:\Users\Administrator\AppData\Local\Temp\ipykernel_8232\98667936.py:22: DeprecationWarning: Use get_tensor_dtype instead.
  dtype = trt.nptype(engine.get_binding_dtype(binding))
C:\Users\Administrator\AppData\Local\Temp\ipykernel_8232\98667936.py:26: DeprecationWarning: Use get_tensor_mode instead.
  if engine.binding_is_input(binding):


True

## Speed Test

In [18]:
%timeit trt_engine.predict(img1,threshold=0.5)

24.7 ms ± 382 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


## formal inference

In [55]:
if os.path.exists("results_trt"):
    shutil.rmtree("results_trt")
os.makedirs("results_trt", exist_ok=True)
trt_engine = BaseEngine("onnx_nms_model/ppyoloe_plus_crn_l_custom_disease_std_v2_nms.engine", imgsz=(512,512))
for img in glob.glob(f"{test_path}/*.jpg"):
    img1 = cv2.imread(img)
    results = trt_engine.predict(img1,threshold=0.4)
    img1 = visualize(img1,results)
    cv2.imwrite("results_trt/"+os.path.basename(img),img1)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_8232\98667936.py:21: DeprecationWarning: Use get_tensor_shape instead.
  size = trt.volume(engine.get_binding_shape(binding))
C:\Users\Administrator\AppData\Local\Temp\ipykernel_8232\98667936.py:22: DeprecationWarning: Use get_tensor_dtype instead.
  dtype = trt.nptype(engine.get_binding_dtype(binding))
C:\Users\Administrator\AppData\Local\Temp\ipykernel_8232\98667936.py:26: DeprecationWarning: Use get_tensor_mode instead.
  if engine.binding_is_input(binding):
